In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import cairocffi
import scanpy as sc
import scipy.io as scio
import csv
import scipy.sparse as sparse
from scipy.sparse import csr_matrix
from scipy.sparse import isspmatrix
from matplotlib.backends.backend_pdf import PdfPages
from functions import * # code by Giuseppe-Antonio Saldi
from sklearn import preprocessing

In [2]:
# functions
def read_betas(t, cardinal_class, boot_path, n_boot=10):
    return {'boot_{}'.format(b): pd.read_table(
            boot_path.format(t=t, b=b),
            index_col=0,header=0) for b in range(1, n_boot+1)}


def B_dot_A(B, A, n_boot):
    acc = B['boot_1']
    for i in range(2, n_boot+1):
        acc += B['boot_{}'.format(i)]
    return acc.dot(A.T) / n_boot

# Hr3 Expression Reconstruction

In [48]:
Hr3_expression_sc = sc.read_h5ad("Reconstruction_input/Hr3_SeuratNormalized_sparse_expression.h5ad")
Hr3_expression_df = Hr3_expression_sc.to_df().T

### MergedDA 

In [49]:
TFA_MergedDA = {'Hr3_MergedDA': pd.read_table("Reconstruction_input//Hr3_ControlAndRNAi_MergedDA_SeuratNormalized_TFA.tsv", index_col=0, header=0)}

In [50]:
# read in betas for the first (0th) task
MergedDA_betas = {}
# t is tasks and b is bootstrap, they will be set in read_betas 
path = 'Reconstruction_input/Lamina_betas/Lamina_MergedDA_Norm2_betas_task_{t}_boot_{b}.tsv'
MergedDA_betas['Zip_MergedDA_betas'] = read_betas(0, 'Zip_MergedDA_betas', path, n_boot=5)

In [51]:
# making sure we have matrices that are compatible on the TF dimension
TFA_MergedDA = TFA_MergedDA['Hr3_MergedDA'][TFA_MergedDA['Hr3_MergedDA'].columns[TFA_MergedDA['Hr3_MergedDA'].columns.isin(MergedDA_betas['Zip_MergedDA_betas']['boot_1'].columns)]]

# computing dot product across 5 bootstraps
MergedDA_avg = B_dot_A(MergedDA_betas['Zip_MergedDA_betas'], TFA_MergedDA, 5)

In [52]:
# sanity checking dimensions
print(MergedDA_avg.shape)

(1682, 1537)


In [53]:
MergedDA_genes_list = MergedDA_avg.index.tolist()

In [54]:
# re-indexing input expression to contain only genes in reconstruction
Hr3_MergedDA_expression_reindexed = Hr3_expression_df[Hr3_expression_df.index.isin(MergedDA_genes_list)]

In [68]:
# sanity check dimensions again
print(Hr3_MergedDA_expression_reindexed.shape)

(1680, 1537)


In [69]:
# re-index reconstructed expression to match input-expression dimensions
Hr3_MergedDA_reconstructed_genes = Hr3_MergedDA_expression_reindexed.index.tolist()

In [70]:
Hr3_MergedDA_reconstructed_reindexed = Hr3_MergedDA_expression_reindexed[Hr3_MergedDA_expression_reindexed.index.isin(Hr3_MergedDA_reconstructed_genes)]

In [71]:
print(Hr3_MergedDA_reconstructed_reindexed.shape)

(1680, 1537)


In [59]:
# save re-indexed input expression
Hr3_MergedDA_expression_reindexed.to_csv("Reconstruction_output/Hr3_inputexpression_MergedDA_reindexed.tsv", sep = '\t')
# save re-indexed reconstructed expression
Hr3_MergedDA_reconstructed_reindexed.to_csv("Reconstruction_output/Hr3_reconstructed_MergedDA_reindexed.tsv", sep = '\t')


### ChromA

In [60]:
TFA_ChromA = {'Hr3_ChromA': pd.read_table("Reconstruction_input/Hr3_ControlAndRNAi_ChromA_SeuratNormalized_TFA.tsv", index_col=0, header=0)}

In [61]:
# read in betas for the first (0th) task
ChromA_betas = {}
# t is tasks and b is bootstrap, they will be set in read_betas 
path = 'Reconstruction_input/Lamina_betas/Lamina_ChromA_Norm2_betas_task_{t}_boot_{b}.tsv'
ChromA_betas['Zip_ChromA_betas'] = read_betas(0, 'Zip_ChromA_betas', path, n_boot=5)

In [62]:
# making sure we have matrices that are compatible on the TF dimension
TFA_ChromA = TFA_ChromA['Hr3_ChromA'][TFA_ChromA['Hr3_ChromA'].columns[TFA_ChromA['Hr3_ChromA'].columns.isin(ChromA_betas['Zip_ChromA_betas']['boot_1'].columns)]]

# computing dot product across 5 bootstraps
ChromA_avg = B_dot_A(ChromA_betas['Zip_ChromA_betas'], TFA_ChromA, 5)

In [63]:
# sanity checking dimensions
print(ChromA_avg.shape)

(1682, 1537)


In [64]:
ChromA_genes_list = ChromA_avg.index.tolist()

In [65]:
# re-indexing input expression to contain only genes in reconstruction
Hr3_ChromA_expression_reindexed = Hr3_expression_df[Hr3_expression_df.index.isin(ChromA_genes_list)]

In [66]:
# sanity check dimensions again
print(Hr3_ChromA_expression_reindexed.shape)

(1680, 1537)


In [72]:
# re-index reconstructed expression to match input-expression dimensions
Hr3_ChromA_reconstructed_genes = Hr3_ChromA_expression_reindexed.index.tolist()

In [73]:
Hr3_ChromA_reconstructed_reindexed = Hr3_ChromA_expression_reindexed[Hr3_ChromA_expression_reindexed.index.isin(Hr3_ChromA_reconstructed_genes)]

In [74]:
print(Hr3_ChromA_reconstructed_reindexed.shape)

(1680, 1537)


In [75]:
# save re-indexed input expression
Hr3_ChromA_expression_reindexed.to_csv("Reconstruction_output/Hr3_inputexpression_ChromA_reindexed.tsv", sep = '\t')
# save re-indexed reconstructed expression
Hr3_ChromA_reconstructed_reindexed.to_csv("Reconstruction_output/Hr3_reconstructed_ChromA_reindexed.tsv", sep = '\t')


### No Bed

In [76]:
TFA_NoBed = {'Hr3_NoBed': pd.read_table("Reconstruction_input/Hr3_ControlAndRNAi_NoBed_SeuratNormalized_TFA.tsv", index_col=0, header=0)}

In [77]:
# read in betas for the first (0th) task
NoBed_betas = {}
# t is tasks and b is bootstrap, they will be set in read_betas 
path = 'Reconstruction_input/Lamina_betas/Lamina_NoBed_Norm2_betas_task_{t}_boot_{b}.tsv'
NoBed_betas['Zip_NoBed_betas'] = read_betas(0, 'Zip_NoBed_betas', path, n_boot=5)

In [78]:
# making sure we have matrices that are compatible on the TF dimension
TFA_NoBed = TFA_NoBed['Hr3_NoBed'][TFA_NoBed['Hr3_NoBed'].columns[TFA_NoBed['Hr3_NoBed'].columns.isin(NoBed_betas['Zip_NoBed_betas']['boot_1'].columns)]]

# computing dot product across 5 bootstraps
NoBed_avg = B_dot_A(NoBed_betas['Zip_NoBed_betas'], TFA_NoBed, 5)

In [79]:
# sanity checking dimensions
print(NoBed_avg.shape)

(1682, 1537)


In [80]:
NoBed_genes_list = NoBed_avg.index.tolist()

In [81]:
# re-indexing input expression to contain only genes in reconstruction
Hr3_NoBed_expression_reindexed = Hr3_expression_df[Hr3_expression_df.index.isin(NoBed_genes_list)]

In [82]:
# sanity check dimensions again
print(Hr3_NoBed_expression_reindexed.shape)

(1680, 1537)


In [83]:
# re-index reconstructed expression to match input-expression dimensions
Hr3_NoBed_reconstructed_genes = Hr3_NoBed_expression_reindexed.index.tolist()

In [84]:
Hr3_NoBed_reconstructed_reindexed = Hr3_NoBed_expression_reindexed[Hr3_NoBed_expression_reindexed.index.isin(Hr3_NoBed_reconstructed_genes)]

In [85]:
print(Hr3_NoBed_reconstructed_reindexed.shape)

(1680, 1537)


In [86]:
# save re-indexed input expression
Hr3_NoBed_expression_reindexed.to_csv("Reconstruction_output/Hr3_inputexpression_NoBed_reindexed.tsv", sep = '\t')
# save re-indexed reconstructed expression
Hr3_NoBed_reconstructed_reindexed.to_csv("Reconstruction_output/Hr3_reconstructed_NoBed_reindexed.tsv", sep = '\t')


# Erm Expression Reconstruction

In [87]:
# read input expression
Erm_expression_sc = sc.read_h5ad("Reconstruction_input/Erm_SeuratNormalized_sparse_expression.h5ad")
Erm_expression_df = Erm_expression_sc.to_df().T

### MergedDA

In [88]:
TFA_MergedDA = {'Erm_MergedDA': pd.read_table("Reconstruction_input/Erm_ControlAndRNAi_MergedDA_SeuratNormalized_TFA.tsv", index_col=0, header=0)}

In [90]:
# read in betas for the first (1st) task
MergedDA_betas = {}
# t is tasks and b is bootstrap, they will be set in read_betas 
# TFs {'ap', 'dve'} removed from betas to align with TFA
path = 'Reconstruction_input/Lamina_betas/Lamina_MergedDA_Norm2_betas_task_{t}_boot_{b}_droppedtfs.tsv'
MergedDA_betas['Des_MergedDA_betas'] = read_betas(1, 'Des_MergedDA_betas', path, n_boot=5)

In [91]:
# making sure we have matrices that are compatible on the TF dimension
TFA_MergedDA = TFA_MergedDA['Erm_MergedDA'][TFA_MergedDA['Erm_MergedDA'].columns[TFA_MergedDA['Erm_MergedDA'].columns.isin(MergedDA_betas['Des_MergedDA_betas']['boot_1'].columns)]]

# computing dot product across 5 bootstraps
MergedDA_avg = B_dot_A(MergedDA_betas['Des_MergedDA_betas'], TFA_MergedDA, 5)

In [92]:
# sanity checking dimensions
print(MergedDA_avg.shape)

(1682, 1000)


In [93]:
MergedDA_genes_list = MergedDA_avg.index.tolist()

In [94]:
# re-indexing input expression to contain only genes in reconstruction
Erm_MergedDA_expression_reindexed = Erm_expression_df[Erm_expression_df.index.isin(MergedDA_genes_list)]

In [95]:
# sanity check dimensions again
print(Erm_MergedDA_expression_reindexed.shape)

(1590, 1000)


In [98]:
# re-index reconstructed expression to match input-expression dimensions
Erm_MergedDA_reconstructed_genes = Erm_MergedDA_expression_reindexed.index.tolist()

In [99]:
Erm_MergedDA_reconstructed_reindexed = Erm_MergedDA_expression_reindexed[Erm_MergedDA_expression_reindexed.index.isin(Erm_MergedDA_reconstructed_genes)]

In [100]:
print(Erm_MergedDA_reconstructed_reindexed.shape)

(1590, 1000)


In [101]:
# save re-indexed input expression
Erm_MergedDA_expression_reindexed.to_csv("Reconstruction_output/Erm_inputexpression_MergedDA_reindexed.tsv", sep = '\t')
# save re-indexed reconstructed expression
Erm_MergedDA_reconstructed_reindexed.to_csv("Reconstruction_output/Erm_reconstructed_MergedDA_reindexed.tsv", sep = '\t')


### ChromA

In [102]:
TFA_ChromA = {'Erm_ChromA': pd.read_table("Reconstruction_input/Erm_ControlAndRNAi_ChromA_SeuratNormalized_TFA.tsv", index_col=0, header=0)}

In [103]:
# read in betas for the first (1st) task
ChromA_betas = {}
# t is tasks and b is bootstrap, they will be set in read_betas 
path = 'Reconstruction_input/Lamina_betas/Lamina_ChromA_Norm2_betas_task_{t}_boot_{b}.tsv'
ChromA_betas['Des_ChromA_betas'] = read_betas(1, 'Des_ChromA_betas', path, n_boot=5)

In [104]:
# making sure we have matrices that are compatible on the TF dimension
TFA_ChromA = TFA_ChromA['Erm_ChromA'][TFA_ChromA['Erm_ChromA'].columns[TFA_ChromA['Erm_ChromA'].columns.isin(ChromA_betas['Des_ChromA_betas']['boot_1'].columns)]]

# computing dot product across 5 bootstraps
ChromA_avg = B_dot_A(ChromA_betas['Des_ChromA_betas'], TFA_ChromA, 5)

In [105]:
# sanity checking dimensions
print(ChromA_avg.shape)

(1682, 1000)


In [106]:
ChromA_genes_list = ChromA_avg.index.tolist()

In [107]:
# re-indexing input expression to contain only genes in reconstruction
Erm_ChromA_expression_reindexed = Erm_expression_df[Erm_expression_df.index.isin(ChromA_genes_list)]

In [108]:
# sanity check dimensions again
print(Erm_ChromA_expression_reindexed.shape)

(1590, 1000)


In [109]:
# re-index reconstructed expression to match input-expression dimensions
Erm_ChromA_reconstructed_genes = Erm_ChromA_expression_reindexed.index.tolist()

In [110]:
Erm_ChromA_reconstructed_reindexed = Erm_ChromA_expression_reindexed[Erm_ChromA_expression_reindexed.index.isin(Erm_ChromA_reconstructed_genes)]

In [111]:
print(Erm_ChromA_reconstructed_reindexed.shape)

(1590, 1000)


In [112]:
# save re-indexed input expression
Erm_ChromA_expression_reindexed.to_csv("Reconstruction_output/Erm_inputexpression_ChromA_reindexed.tsv", sep = '\t')
# save re-indexed reconstructed expression
Erm_ChromA_reconstructed_reindexed.to_csv("Reconstruction_output/Erm_reconstructed_ChromA_reindexed.tsv", sep = '\t')


### No Bed

In [113]:
TFA_NoBed = {'Erm_NoBed': pd.read_table("Reconstruction_input/Erm_ControlAndRNAi_NoBed_SeuratNormalized_TFA.tsv", index_col=0, header=0)}

In [114]:
# read in betas for the first (0th) task
NoBed_betas = {}
# t is tasks and b is bootstrap, they will be set in read_betas 
# TFs {'E(spl)m7-HLH', 'E(spl)m8-HLH', 'bsh'} removed from betas to align with TFA
path = 'Reconstruction_input/Lamina_betas/Lamina_NoBed_Norm2_betas_task_{t}_boot_{b}_droppedtfs.tsv'
NoBed_betas['Des_NoBed_betas'] = read_betas(1, 'Des_NoBed_betas', path, n_boot=5)

In [115]:
# making sure we have matrices that are compatible on the TF dimension
TFA_NoBed = TFA_NoBed['Erm_NoBed'][TFA_NoBed['Erm_NoBed'].columns[TFA_NoBed['Erm_NoBed'].columns.isin(NoBed_betas['Des_NoBed_betas']['boot_1'].columns)]]

# computing dot product across 5 bootstraps
NoBed_avg = B_dot_A(NoBed_betas['Des_NoBed_betas'], TFA_NoBed, 5)

In [116]:
# sanity checking dimensions
print(NoBed_avg.shape)

(1682, 1000)


In [117]:
NoBed_genes_list = NoBed_avg.index.tolist()

In [118]:
# re-indexing input expression to contain only genes in reconstruction
Erm_NoBed_expression_reindexed = Erm_expression_df[Erm_expression_df.index.isin(NoBed_genes_list)]

In [119]:
# sanity check dimensions again
print(Erm_NoBed_expression_reindexed.shape)

(1590, 1000)


In [120]:
# re-index reconstructed expression to match input-expression dimensions
Erm_NoBed_reconstructed_genes = Erm_NoBed_expression_reindexed.index.tolist()

In [121]:
Erm_NoBed_reconstructed_reindexed = Erm_NoBed_expression_reindexed[Erm_NoBed_expression_reindexed.index.isin(Erm_NoBed_reconstructed_genes)]

In [122]:
print(Erm_NoBed_reconstructed_reindexed.shape)

(1590, 1000)


In [123]:
# save re-indexed input expression
Erm_NoBed_expression_reindexed.to_csv("Reconstruction_output/Erm_inputexpression_NoBed_reindexed.tsv", sep = '\t')
# save re-indexed reconstructed expression
Erm_NoBed_reconstructed_reindexed.to_csv("Reconstruction_output/Erm_reconstructed_NoBed_reindexed.tsv", sep = '\t')


# Pdm3 Expression Reconstruction

In [125]:
# read input expression
pdm3_expression_sc = sc.read_h5ad("Reconstruction_input/pdm3_SeuratNormalized_sparse_expression.h5ad")
pdm3_expression_df = pdm3_expression_sc.to_df().T

### MergedDA

In [126]:
TFA_MergedDA = {'pdm3_MergedDA': pd.read_table("Reconstruction_input/Pdm3_ControlAndRNAi_MergedDA_SeuratNormalized_TFA.tsv", index_col=0, header=0)}

In [127]:
# read in betas for the first (0th) task
MergedDA_betas = {}
# t is tasks and b is bootstrap, they will be set in read_betas 
path = 'Reconstruction_input/Tm_betas/Tm_MergedDA_norm2_betas_task_{t}_boot_{b}.tsv'
MergedDA_betas['Zip_MergedDA_betas'] = read_betas(0, 'Zip_MergedDA_betas', path, n_boot=5)

In [128]:
# making sure we have matrices that are compatible on the TF dimension
TFA_MergedDA = TFA_MergedDA['pdm3_MergedDA'][TFA_MergedDA['pdm3_MergedDA'].columns[TFA_MergedDA['pdm3_MergedDA'].columns.isin(MergedDA_betas['Zip_MergedDA_betas']['boot_1'].columns)]]

# computing dot product across 5 bootstraps
MergedDA_avg = B_dot_A(MergedDA_betas['Zip_MergedDA_betas'], TFA_MergedDA, 5)

In [129]:
# sanity checking dimensions
print(MergedDA_avg.shape)

(1394, 1779)


In [130]:
MergedDA_genes_list = MergedDA_avg.index.tolist()

In [131]:
# re-indexing input expression to contain only genes in reconstruction
pdm3_MergedDA_expression_reindexed = pdm3_expression_df[pdm3_expression_df.index.isin(MergedDA_genes_list)]

In [132]:
# sanity check dimensions again
print(pdm3_MergedDA_expression_reindexed.shape)

(1392, 1779)


In [133]:
# re-index reconstructed expression to match input-expression dimensions
pdm3_MergedDA_reconstructed_genes = pdm3_MergedDA_expression_reindexed.index.tolist()

In [134]:
pdm3_MergedDA_reconstructed_reindexed = pdm3_MergedDA_expression_reindexed[pdm3_MergedDA_expression_reindexed.index.isin(pdm3_MergedDA_reconstructed_genes)]

In [135]:
print(pdm3_MergedDA_reconstructed_reindexed.shape)

(1392, 1779)


In [136]:
# save re-indexed input expression
pdm3_MergedDA_expression_reindexed.to_csv("Reconstruction_output/pdm3_inputexpression_MergedDA_reindexed.tsv", sep = '\t')
# save re-indexed reconstructed expression
pdm3_MergedDA_reconstructed_reindexed.to_csv("Reconstruction_output/pdm3_reconstructed_MergedDA_reindexed.tsv", sep = '\t')


### ChromA

In [137]:
TFA_ChromA = {'Pdm3_ChromA': pd.read_table("Reconstruction_input/Pdm3_ControlAndRNAi_ChromA_SeuratNormalized_TFA.tsv", index_col=0, header=0)}

In [138]:
# read in betas for the first (0th) task
ChromA_betas = {}
# t is tasks and b is bootstrap, they will be set in read_betas 
path = 'Reconstruction_input/Tm_betas/Tm_ChromA_norm2_betas_task_{t}_boot_{b}.tsv'
ChromA_betas['Zip_ChromA_betas'] = read_betas(0, 'Zip_ChromA_betas', path, n_boot=5)

In [139]:
# making sure we have matrices that are compatible on the TF dimension
TFA_ChromA = TFA_ChromA['Pdm3_ChromA'][TFA_ChromA['Pdm3_ChromA'].columns[TFA_ChromA['Pdm3_ChromA'].columns.isin(ChromA_betas['Zip_ChromA_betas']['boot_1'].columns)]]

# computing dot product across 5 bootstraps
ChromA_avg = B_dot_A(ChromA_betas['Zip_ChromA_betas'], TFA_ChromA, 5)

In [140]:
# sanity checking dimensions
print(ChromA_avg.shape)

(1394, 1779)


In [141]:
ChromA_genes_list = ChromA_avg.index.tolist()

In [142]:
# re-indexing input expression to contain only genes in reconstruction
pdm3_ChromA_expression_reindexed = pdm3_expression_df[pdm3_expression_df.index.isin(ChromA_genes_list)]

In [143]:
# sanity check dimensions again
print(pdm3_ChromA_expression_reindexed.shape)

(1392, 1779)


In [144]:
# re-index reconstructed expression to match input-expression dimensions
pdm3_ChromA_reconstructed_genes = pdm3_ChromA_expression_reindexed.index.tolist()

In [145]:
pdm3_ChromA_reconstructed_reindexed = pdm3_ChromA_expression_reindexed[pdm3_ChromA_expression_reindexed.index.isin(pdm3_ChromA_reconstructed_genes)]

In [146]:
print(pdm3_ChromA_reconstructed_reindexed.shape)

(1392, 1779)


In [147]:
# save re-indexed input expression
pdm3_ChromA_expression_reindexed.to_csv("Reconstruction_output/pdm3_inputexpression_ChromA_reindexed.tsv", sep = '\t')
# save re-indexed reconstructed expression
pdm3_ChromA_reconstructed_reindexed.to_csv("Reconstruction_output/pdm3_reconstructed_ChromA_reindexed.tsv", sep = '\t')


### No Bed

In [148]:
TFA_NoBed = {'pdm3_NoBed': pd.read_table("Reconstruction_input/Pdm3_ControlAndRNAi_NoBed_SeuratNormalized_TFA.tsv", index_col=0, header=0)}

In [149]:
# read in betas for the first (0th) task
NoBed_betas = {}
# t is tasks and b is bootstrap, they will be set in read_betas 
path = 'Reconstruction_input/Tm_betas/Tm_NoBed_norm2_betas_task_{t}_boot_{b}.tsv'
NoBed_betas['Zip_NoBed_betas'] = read_betas(0, 'Zip_NoBed_betas', path, n_boot=5)

In [150]:
# making sure we have matrices that are compatible on the TF dimension
TFA_NoBed = TFA_NoBed['pdm3_NoBed'][TFA_NoBed['pdm3_NoBed'].columns[TFA_NoBed['pdm3_NoBed'].columns.isin(NoBed_betas['Zip_NoBed_betas']['boot_1'].columns)]]

# computing dot product across 5 bootstraps
NoBed_avg = B_dot_A(NoBed_betas['Zip_NoBed_betas'], TFA_NoBed, 5)

In [151]:
# sanity checking dimensions
print(NoBed_avg.shape)

(1394, 1779)


In [152]:
NoBed_genes_list = NoBed_avg.index.tolist()

In [153]:
# re-indexing input expression to contain only genes in reconstruction
pdm3_NoBed_expression_reindexed = pdm3_expression_df[pdm3_expression_df.index.isin(NoBed_genes_list)]

In [154]:
# sanity check dimensions again
print(pdm3_NoBed_expression_reindexed.shape)

(1392, 1779)


In [155]:
# re-index reconstructed expression to match input-expression dimensions
pdm3_NoBed_reconstructed_genes = pdm3_NoBed_expression_reindexed.index.tolist()

In [156]:
pdm3_NoBed_reconstructed_reindexed = pdm3_NoBed_expression_reindexed[pdm3_NoBed_expression_reindexed.index.isin(pdm3_NoBed_reconstructed_genes)]

In [157]:
print(pdm3_NoBed_reconstructed_reindexed.shape)

(1392, 1779)


In [158]:
# save re-indexed input expression
pdm3_NoBed_expression_reindexed.to_csv("Reconstruction_output/pdm3_inputexpression_NoBed_reindexed.tsv", sep = '\t')
# save re-indexed reconstructed expression
pdm3_NoBed_reconstructed_reindexed.to_csv("Reconstruction_output/pdm3_reconstructed_NoBed_reindexed.tsv", sep = '\t')
